<a href="https://colab.research.google.com/github/HEON-WOOK/Portfolio2/blob/main/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [3]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy() 

In [10]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size = 0.2, random_state=42)


In [11]:
sub_input,val_input,sub_target,val_target = train_test_split(train_input,train_target,test_size=0.2,random_state= 42)


In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit (sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [19]:
### K-겹 교차검증 (Defalut값 = 5겹)

from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(score)

{'fit_time': array([0.00754976, 0.00711656, 0.00734591, 0.0076406 , 0.00696802]), 'score_time': array([0.00082612, 0.00060606, 0.00058651, 0.00064254, 0.0005722 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [20]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [21]:
scores['test_score']

array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])

In [22]:
### k폴드 시 데이터 섞기
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input,train_target)
print(np.mean(scores['test_score']))

0.855300214703487


In [23]:
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [28]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),'max_depth':range(5,20,1),'min_samples_split':range(2,100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [29]:
from scipy.stats import uniform,randint

In [38]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),'max_depth':randint(20,50),'min_samples_split':randint(2,25),'min_samples_leaf':randint(1,25)}

In [41]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [42]:
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input,test_target))

0.8695428296438884
0.86


In [44]:
np.mean(gs.cv_results_['mean_test_score'])

0.8639124620567113